In [1]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
import matplotlib.pyplot as plt

In [48]:
length_ = 10 # length of each screen
width_ = 10  # width of each screen
dist_ = 2 # distance between the screens
h = 0.1 # grid size

In [49]:
corners1 = np.array([ [-0.5*width_, -0.5*length_, 0.5*dist_],
                     [-0.5*width_,  0.5*length_, 0.5*dist_],
                     [0.5*width_,   0.5*length_, 0.5*dist_],
                     [0.5*width_,  -0.5*length_, 0.5*dist_]])
corners2 = np.array([ [-0.5*width_, -0.5*length_, -0.5*dist_],
                     [-0.5*width_,  0.5*length_, -0.5*dist_],
                     [0.5*width_,   0.5*length_, -0.5*dist_],
                     [0.5*width_,  -0.5*length_, -0.5*dist_]])
grid1 = bempp.api.shapes.screen(corners1, h = h)
grid2 = bempp.api.shapes.screen(corners2, h = h)

In [50]:
rwg1 = bempp.api.function_space(grid1, "RWG", 0)
rwg2 = bempp.api.function_space(grid2, "RWG", 0)

snc1 = bempp.api.function_space(grid1, "SNC", 0)
snc2 = bempp.api.function_space(grid2, "SNC", 0)

In [51]:
N_wn = 15
lbound_wn = 0.0008
ubound_wn = 20
quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)
results = np.empty(N_wn, dtype='float64')

In [ ]:
for index, point in enumerate(quad_points):

    wavenumber = -1j * np.log(point)

    op11 = bempp.api.operators.boundary.maxwell.electric_field(rwg1, rwg1, snc1, wavenumber)
    op22 = bempp.api.operators.boundary.maxwell.electric_field(rwg2, rwg2, snc2, wavenumber)
    op21 = bempp.api.operators.boundary.maxwell.electric_field(rwg1, rwg2, snc2, wavenumber)
    op12 = bempp.api.operators.boundary.maxwell.electric_field(rwg2, rwg1, snc1, wavenumber)

    mat11 = op11.weak_form().A
    mat12 = op12.weak_form().A
    mat21 = op21.weak_form().A
    mat22 = op22.weak_form().A
    mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
    mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

    mat = np.real(np.block([[mat11,mat12],[mat21,mat22]]))    

    mat11_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat11), np.eye(mat11.shape[0]))
    mat22_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat22), np.eye(mat22.shape[0]))

    inv_free_mat2 = np.block([[mat11_inv,  mat12_zero],
                          [mat21_zero, mat22_inv]])  

    combined_inverse_free = inv_free_mat2.dot(mat)
    sign, logdet = slogdet(combined_inverse_free)
    
    results[index] = logdet
    print(f"Log Determinant: {index}: {logdet}")

In [ ]:
%matplotlib inline
from matplotlib.pyplot import figure

fig = figure()
ax1 = fig.add_subplot(121)
ax1.set_xlabel('1j * k')
ax2 = fig.add_subplot(122)
ax2.set_xlabel('exp(-k)')
ax1.semilogy(-np.log(quad_points), -results)
ax2.plot(quad_points, -results)
fig.tight_layout()

In [ ]:
casimir_energy = np.trapz(-results / quad_points, quad_points) / (2 * np.pi)
print(f"The normalized Casimir Energy is: {casimir_energy}")

area_ = length_ * width_
casimir_energy_unit_area = casimir_energy/area_
print(f"The normalized Casimir Energy per unit area is: {casimir_energy_unit_area}")